In [ ]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra
using Distributions
using Statistics
using Dates
using Plots
ENV["COLUMNS"] = 200;
include("COVIDResourceAllocation.jl")
using .COVIDResourceAllocation

In [ ]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 3, 30)
end_date   = Date(2020, 6, 30)
travel_threshold_hours = 4.0

adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [ ]:
hospitalized_days = 14
forecast_initial = forecast(
    states, start_date-Dates.Day(1), start_date-Dates.Day(1),
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
)[:]
forecast_active = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_admitted = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = forecast(
    states, start_date-Dates.Day(hospitalized_days), end_date-Dates.Day(hospitalized_days),
    level=:state,
    source=:ihme,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
);

In [ ]:
pct_nurses_available = 0.5
patients_per_nurse_covid = 2.5
nurse_hrs_per_week_covid = 36
#pct_beds_available = 0.25

#beds = n_beds(states, bed_type=:all, pct_beds_available=pct_beds_available)

N = length(states);
T = (end_date - start_date).value + 1

#forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days))

@show nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7
@show nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid

@show nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid;

nurses = n_nurses(states) * Float32(pct_nurses_available);

In [63]:
isolation_spot = vec([0,0,0,0,0,0,1,1,0,0,0])

model = patient_nurse_allocation(
    forecast_initial,
    forecast_discharged,
    forecast_admitted,
    nurses,
    adj,
    #isolation_spot,
    los = hospitalized_days,
    nurse_days_per_patient_day = nurse_days_per_patient_day_covid,
    sent_penalty = 0,
    no_artificial_shortage = false,
    disallow_nurse_shortage_sent = false,
    disallow_nurse_shortage_newpatients = false,
    verbose=true
)
sentn = value.(model[:sentnurses])
sentp = value.(model[:sentpatients])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

In [ ]:
results_old = PatientAllocationResults.results_all(sentp, nurses, forecast_initial, forecast_discharged,forecast_admitted, states, start_date, hospitalized_days);

In [ ]:
results_old.sent_matrix_table

In [ ]:
demand_old = results.active_patients * nurse_days_per_patient_day_covid;

In [ ]:
results_nurses_old = NurseAllocationResults.results_all(sentn, nurses, demand, states, start_date);

In [ ]:
results_nurses_old.summary_table

In [ ]:
results_nurses_old.average_load

In [ ]:
results_nurses.sent_matrix_table

In [ ]:
results.sent_to

## Plotting overflow over time for historical data

In [ ]:
# note I exlcuded the initial patients
net_patients_per_day = forecast_active-forecast_discharged+forecast_admitted;

In [ ]:
nurses_p_day = (1/nurse_days_per_patient_day_covid)*pct_nurses_available*Matrix(DataFrame(repeat(nurses, 1, size(net_patients_per_day)[2]), :auto))
beds_p_day = pct_beds_available*Matrix(DataFrame(repeat(beds, 1, size(net_patients_per_day)[2]), :auto))
capacity = nurses_p_day+beds_p_day;

In [ ]:
start_date = Date(2020, 3, 30)
end_date   = Date(2020, 6, 30)
date_range = start_date:end_date
overflow = net_patients_per_day - capacity
overflows_p_hosp = vec(sum(overflow, dims=1));

In [ ]:
date_range = start_date:end_date
gr()
plot(date_range, overflows_p_hosp,
     xlabel = "Date",
     ylabel = "Cumulative Overflow",
     title = "Overflow over Time",
     legend = false,
     linewidth = 2,fmt=:png) 


In [ ]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
states_cleaned = [replace(state, r"[, ]" => "") for state in states]
states_cleaned_row = hcat(states_cleaned...)

plot(date_range, [vec(overflow[i,:]) for i in 1:size(overflow)[1]],
     xlabel = "Date",
     ylabel = "Overflow By Hospital",
     title = "Overflow over Time",
     legend = :topright,
     label = states_cleaned_row,
     linewidth = 2,
     fmt=:png)
savefig(joinpath("../figures", "Historical_overflow_plot.png"))

In [ ]:
# total overflow overtime regardless of states
df = results.complete_table
cleaned_df = DataFrame(Date=df.date, Sent=df.sent)
# Group by state and date, then create the 11x93 matrix
grouped_df = groupby(cleaned_df, :Date)

In [ ]:
df = results.complete_table
cleaned_df = DataFrame(Date=df.date, Sent=df.sent)
# Group by state and date, then create the 11x93 matrix
grouped_df = groupby(cleaned_df, :Date)

# check the shortage matrix
#size(shortage_matrix)
df = results.complete_table
cleaned_df = DataFrame(State=df.location, Date=df.date, Sent=df.sent)
# Group by state and date, then create the 11x93 matrix
grouped_df = groupby(cleaned_df, :State)
sent_matrix = zeros(11, 93)
for (i, state_df) in enumerate(grouped_df)
    sent_matrix[i, :] .= state_df.Sent
end

plot(date_range, [vec(grouped_df[i]) for i in 1:length(grouped_df)],
     xlabel = "Date",
     ylabel = "Patient Transfer By Hospital",
     title = "Total Patient Transfer Over Time",
     legend = :topright,
     label = states_cleaned_row,
     linewidth = 2,
     fmt=:png)
#savefig(joinpath("../figures", "PNA_PSent_plot.png"))

## Plotting Overflow Overtime for Transfer Model

Plot Patients Sent over Time

In [ ]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
states_cleaned = [replace(state, r"[, ]" => "") for state in states]
states_cleaned_row = hcat(states_cleaned...)

df = results.complete_table
cleaned_df = DataFrame(State=df.location, Date=df.date, Sent=df.sent)
# Group by state and date, then create the 11x93 matrix
grouped_df = groupby(cleaned_df, :State)
sent_matrix = zeros(11, 93)
for (i, state_df) in enumerate(grouped_df)
    sent_matrix[i, :] .= state_df.Sent
end

# check the shortage matrix
#size(shortage_matrix)

plot(date_range, [vec(sent_matrix[i,:]) for i in 1:size(sent_matrix)[1]],
     xlabel = "Date",
     ylabel = "Patient Transfer By Hospital",
     title = "Total Patient Transfer Over Time",
     legend = :topright,
     label = states_cleaned_row,
     linewidth = 2,
     fmt=:png)
savefig(joinpath("../figures", "PNA_PSent_plot.png"))

## Shortage Plots

In [ ]:
net_shortage = net_patients_per_day*nurse_days_per_patient_day_covid - nurses_p_day*pct_nurses_available;
net_shortage_p_hosp = vec(sum(net_shortage, dims=1));

In [ ]:
# smoothed demand
net_shortage = demand - nurses_p_day*pct_nurses_available;
net_shortage_p_hosp = vec(sum(net_shortage, dims=1));

In [ ]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
states_cleaned = [replace(state, r"[, ]" => "") for state in states]
states_cleaned_row = hcat(states_cleaned...)

plot(date_range, [vec(net_shortage[i,:]) for i in 1:size(net_shortage)[1]],
     xlabel = "Date",
     ylabel = "Shortage By Hospital",
     title = "Shortage over Time",
     legend = :topright,
     label = states_cleaned_row,
     linewidth = 2,
     fmt=:png)
savefig(joinpath("../figures", "Historical_shortage_plot.png"))

Plot shortage over time using patient-nurse allocaiton model (without isolation spots)

In [ ]:
start_date = Date(2020, 3, 30)
end_date   = Date(2020, 6, 30)
date_range = start_date:end_date
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
states_cleaned = [replace(state, r"[, ]" => "") for state in states]
states_cleaned_row = hcat(states_cleaned...)
df = results_nurses_old.complete_table
cleaned_df = DataFrame(State=df.state, Date=df.date, Shortage=df.shortage)
# Group by state and date, then create the 11x93 matrix
grouped_df = groupby(cleaned_df, :State)
shortage_matrix = zeros(11, 93)
for (i, state_df) in enumerate(grouped_df)
    shortage_matrix[i, :] .= state_df.Shortage
end

# check the shortage matrix
#size(shortage_matrix)

net_shortage = shortage_matrix

plot(date_range, [vec(net_shortage[i,:]) for i in 1:size(net_shortage)[1]],
     xlabel = "Date",
     ylabel = "Shortage By Hospital",
     title = "Shortage over Time",
     legend = :topright,
     label = states_cleaned_row,
     linewidth = 2,
     fmt=:png)
#savefig(joinpath("../figures", "PNA_shortage_plot_disallow_shortage_sent.png"))
     

In [ ]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
states_cleaned = [replace(state, r"[, ]" => "") for state in states]
states_cleaned_row = hcat(states_cleaned...)

df = results_nurses.complete_table
cleaned_df = DataFrame(State=df.state, Date=df.date, Sent=df.sent)
# Group by state and date, then create the 11x93 matrix
grouped_df = groupby(cleaned_df, :State)
sent_matrix = zeros(11, 93)
for (i, state_df) in enumerate(grouped_df)
    sent_matrix[i, :] .= state_df.Sent
end

# check the shortage matrix
#size(shortage_matrix)

plot(date_range, [vec(sent_matrix[i,:]) for i in 1:size(sent_matrix)[1]],
     xlabel = "Date",
     ylabel = "Nurse Transfer By Hospital",
     title = "Total Nurse Transfer Over Time",
     legend = :topright,
     label = states_cleaned_row,
     linewidth = 2,
     fmt=:png)
savefig(joinpath("../figures", "PNA_NSent_plot.png"))